不管是OpenAI还是Claude等AI厂商，都对外提供大模型的API调用服务，允许开发者调用大模型以完成任务。

> 下面例子以DeepSeek模型为例

## API文档
和DeepSeek API分为Chat和Completion两种。
其中Chat具有支持多轮对话、多种角色、上下文管理的特性，适合完成例如聊天机器人、多轮任务写作等功能。
Completion专注于一次性任务的完成。更加适合文本生成、翻译等任务。

下面是Chat对话补全的Demo(非流式输出)

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionSystemMessageParam, ChatCompletionUserMessageParam, ChatCompletionAssistantMessageParam

load_dotenv()

llm = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)
response = llm.chat.completions.create(
    model="deepseek-chat",
    messages=[
        ChatCompletionSystemMessageParam(role="system", content="You are a useful coder"),
        ChatCompletionUserMessageParam(role="user", content="Please write quick sort code"),
        ChatCompletionAssistantMessageParam(role="assistant", content="Please generate by python")
    ],
    max_tokens=1024,
    temperature=0.7,
    stream=True
)

print(response.choices[0].message.content)

如果需要流式输出，可以使用下面的方式获取

In [ ]:
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

如果要使用代码补全功能，建议使用Completion，下面是使用DeepSeek Completion方式完成

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

llm = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://api.deepseek.com/beta"
)

response = llm.completions.create(
    model="deepseek-chat",
    prompt="def fib(a):",
    suffix="    return fib(a-1) + fib(a-2)",
    max_tokens=128
)

print(response.choices[0].text)

## API指南

### 思考模式

在输出最终答案前，模型会先输出思维链，用于提升结果的准确性。

需要注意的是：
1. 思维链也是属于输出内容，会占用output max token。
2. response对象的reasoning_content是思维链的内容，而content是最终的答案。
3. 对于多轮对话，当轮的上下文中不会包含reasoning_content内容，而只包括content内容。

样例Demo:

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)

# Turn 1
messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)

step01_content = ""
for chunk in response:
    if chunk.choices[0].delta.reasoning_content:
        print(chunk.choices[0].delta.reasoning_content, end="", flush=True)
    else:
        step01_content += chunk.choices[0].delta.content

print(f"step01 content\n {step01_content}")
# Turn 2
messages.append({"role": "assistant", "content": step01_content})
messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)

step02_content = ""

for chunk in response:
    if chunk.choices[0].delta.reasoning_content:
        print(chunk.choices[0].delta.reasoning_content, end="", flush=True)
    else:
        step02_content += chunk.choices[0].delta.content

print(f"step02 content\n {step02_content}")